In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dku-data-science/sample_submission_final.csv
/kaggle/input/dku-data-science/train.csv
/kaggle/input/dku-data-science/test.csv


# 0️⃣ 필수 라이브리리

In [2]:
# 사용 라이브러리
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.utils.class_weight import compute_class_weight # 가중치 부여

import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance

from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, auc
import xgboost as xgb

# 1️⃣ EDA

In [3]:
# 데이터셋 불러기기
base_path = "/kaggle/input/dku-data-science"

train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))
submission_df = pd.read_csv(os.path.join(base_path, "sample_submission_final.csv"))

In [4]:
train_df.head()

,module(equipment),timestamp,localtime,operation,voltageR,voltageS,voltageT,voltageRS,voltageST,voltageTR,currentR,currentS,currentT,activePower,powerFactorR,powerFactorS,powerFactorT,reactivePowerLagging,accumActiveEnergy
0,14(2호기),1739918155000,20250218143555,1,216.65,215.21,216.89,373.99,374.20,375.45,17.76,6.59,10.99,2039.66,91.24,93.67,88.05,369.96,8789565
1,16(호이스트),1742552385000,20250321031945,1,212.13,216.51,218.88,371.20,377.04,373.26,17.75,12.07,10.94,2339.27,94.75,88.46,93.38,423.29,9147762
2,11(우측분전반1),1738245780000,20250130060300,1,219.30,211.36,212.71,372.96,367.24,374.12,27.54,12.88,24.68,3736.34,98.70,87.36,88.44,490.54,5480746
3,5(좌측분전반),1740645790000,20250227004310,1,214.15,213.73,210.73,370.54,367.58,367.95,26.78,7.03,22.33,3185.26,98.62,90.41,92.84,466.56,9190918
4,18(우측분전반2),1737829500000,20250125102500,1,212.07,215.52,215.66,370.29,373.40,370.41,29.51,8.88,6.82,2571.53,85.52,90.74,97.44,392.70,8606780


## 컬럼별 유형 확인
- module만 범주형
- 나머지는 전부 수치형 데이터

In [5]:
# train data 컬럼 별 유형 확인
print('train_df information')
print(train_df.info())
print()

# test data 컬럼 별 유형 확인
print('test_df information')
print(test_df.info())

train_df information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23587209 entries, 0 to 23587208
Data columns (total 19 columns):
 #   Column                Dtype  
---  ------                -----  
 0   module(equipment)     object 
 1   timestamp             int64  
 2   localtime             int64  
 3   operation             int64  
 4   voltageR              float64
 5   voltageS              float64
 6   voltageT              float64
 7   voltageRS             float64
 8   voltageST             float64
 9   voltageTR             float64
 10  currentR              float64
 11  currentS              float64
 12  currentT              float64
 13  activePower           float64
 14  powerFactorR          float64
 15  powerFactorS          float64
 16  powerFactorT          float64
 17  reactivePowerLagging  float64
 18  accumActiveEnergy     int64  
dtypes: float64(14), int64(4), object(1)
memory usage: 3.3+ GB
None

test_df information
<class 'pandas.core.frame.DataFrame'>
Ra

## 결측치 여부 확인
- 결측치 하나도 없음

In [6]:
# 결측치 여부 및 개수 확인
print("train_df 결측치 현황")
print(train_df.isnull().sum())

print("\n test_df 결측치 현황")
print(test_df.isnull().sum())

train_df 결측치 현황
module(equipment)       0
timestamp               0
localtime               0
operation               0
voltageR                0
voltageS                0
voltageT                0
voltageRS               0
voltageST               0
voltageTR               0
currentR                0
currentS                0
currentT                0
activePower             0
powerFactorR            0
powerFactorS            0
powerFactorT            0
reactivePowerLagging    0
accumActiveEnergy       0
dtype: int64

 test_df 결측치 현황
module(equipment)       0
timestamp               0
localtime               0
operation               0
voltageR                0
voltageS                0
voltageT                0
voltageRS               0
voltageST               0
voltageTR               0
currentR                0
currentS                0
currentT                0
activePower             0
powerFactorR            0
powerFactorS            0
powerFactorT            0
reactivePowerLaggi

## 컬럼별 고유값 수

In [7]:
print("train_df 컬럼별 고유값 수")
print(train_df.nunique())

print("\n test_df 컬럼별 고유값 수")
print(test_df.nunique())

train_df 컬럼별 고유값 수
module(equipment)             13
timestamp                2591281
localtime                2592001
operation                      1
voltageR                    2755
voltageS                    2790
voltageT                    2780
voltageRS                   3900
voltageST                   3917
voltageTR                   3889
currentR                    2501
currentS                    2501
currentT                    2501
activePower               402760
powerFactorR                4001
powerFactorS                4001
powerFactorT                4001
reactivePowerLagging      132977
accumActiveEnergy       11164883
dtype: int64

 test_df 컬럼별 고유값 수
module(equipment)            13
timestamp               2566100
localtime               2566802
operation                     1
voltageR                   2486
voltageS                   2505
voltageT                   2508
voltageRS                  3443
voltageST                  3437
voltageTR                  3414
c

# 2️⃣데이터 시각화

# 3️⃣ 데이터 전처리

In [8]:
# 시간 정보 처리
for df in [train_df, test_df]:
    df['datetime'] = pd.to_datetime(df['localtime'], format="%Y%m%d%H%M%S")
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month

# 4️⃣ 모델 학습

In [9]:
# 사용할 특징(예: 전류, 전압, 역률 등)
features = ['voltageR', 'voltageS', 'voltageT',
            'currentR', 'currentS', 'currentT',
            'powerFactorR', 'powerFactorS', 'powerFactorT',
            'reactivePowerLagging', 'hour', 'day', 'month']

target = 'activePower'  # hourly_pow 대신 activePower을 시간 단위로 해석

# 학습 데이터
X_train = train_df[features]
y_train = train_df[target]

# 테스트 데이터
X_test = test_df[features]

In [10]:
model = xgb.XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# 5️⃣ 모델 검증

# 6️⃣ 제출

In [11]:
# hourly_pow 예측
test_df['hourly_pow'] = model.predict(X_test)

# 누적 전력량
test_df['agg_pow'] = test_df['hourly_pow'].cumsum()

# 요금 및 탄소배출량 계산
test_df['may_bill'] = test_df['hourly_pow'] * 110         # 단가 110원/kWh
test_df['may_carbon'] = test_df['hourly_pow'] * 0.424 

In [12]:
# 샘플 제출 양식 불러오기
submission = pd.read_csv("/kaggle/input/dku-data-science/sample_submission_final.csv")

# datetime을 id로 맞추기
test_df['id'] = test_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

# 필요한 컬럼만 정리
submission = test_df[['id', 'hourly_pow', 'agg_pow', 'may_bill', 'may_carbon']]

# 저장
submission.to_csv("/kaggle/working/submission.csv", index=False)